In [1]:
import os
import json

def build_label_dict():
    d = {}
    label_path = r"./Frames-dataset/labels.txt"
    label_file = open(label_path, encoding='utf-8')
    for line in label_file:
        k, v = line.split(",")
        d[k] = True if v.strip()=="True" else False
    return d


In [2]:
import rdflib
import os, sys

# text = sys.argv[1]

def load_graph():
    g = rdflib.Graph()
    g.parse("onto.nt", format="nt")
    return g

def check_in_ontology(g, a, b):
    q = "select * where {<http://o.org/"+a+"> ?p <http://o.org/"+b+">}"
    x1 = g.query(q)
    for x in x1:
        print()
        print(a,x,b)
        get_type(g, a)
        get_type(g, b)
    
    return x1


def get_type(g, a):
    q = "select ?t where {<http://o.org/"+a+"> <http://o.org/type> ?t}"
    x1 = g.query(q)
    for x in x1:
        print(a,x)


def normalize_word(w):
    # TODO: stemming
    return w.replace(",","").replace(".","").strip()
    
def check_text(g, text):
    text = text.split()
    for t1 in text:
        t1 = normalize_word(t1)
        for t2 in text:
            t2 = normalize_word(t2)
            if t1 != t2:
                check_in_ontology(g, t1, t2)
                
def geographic_difference(g, s, t):
    # Currently - equal weight for every connection
    s = s.lower()
    t = t.lower()
    if s == t:
        return 0
    return 3 - len(check_in_ontology(g, s, t))

def month_to_num(m):
    if m == "jan":
        return 1
    if m == "feb":
        return 2
    if m == "mar":
        return 3
    if m == "apr":
        return 4
    if m == "may":
        return 5
    if m == "jun":
        return 6
    if m == "jul":
        return 7
    if m == "aug":
        return 8
    if m == "sep":
        return 9
    if m == "oct":
        return 10
    if m == "nov":
        return 11
    if m == "dec":
        return 12
    

def date_difference(s, t):
    s_splitted = s.lower().split()
    t_splitted = t.lower().split()
    s_m = month_to_num(s_splitted[0][0:3])
    s_d = s_splitted[1]
    t_m = month_to_num(t_splitted[0][0:3])
    t_d = t_splitted[1]
    
    #print(s_m, s_d, t_m, t_d)

    return abs((s_m*30+int(s_d)) - (t_m*30+int(t_d)))

def prepare_frames_vector(chat,g):
    dst_city = or_city = str_date = end_date = ""
    or_city_diff = dst_city_diff = str_date_diff = end_date_diff = 0
    for t in chat["turns"]:
        if "text" in t:
            prev_or_city = or_city
            prev_dst_city = dst_city
            prev_str_date = str_date
            prev_end_date = end_date
            
            for f in t["labels"]["frames"]:
                #print(f["info"])
                info = f["info"]
                if "or_city" in info:
                    or_city = info["or_city"][0]["val"]
                if "dst_city" in info:
                    dst_city = info["dst_city"][0]["val"]
                if "str_date" in info:
                    str_date = info["str_date"][0]["val"]
                if "end_date" in info:
                    end_date = info["end_date"][0]["val"]

            if len(prev_or_city) > 1:
                or_city_diff = geographic_difference(g, prev_or_city, or_city)
            if len(prev_dst_city) > 1:
                dst_city_diff = geographic_difference(g, prev_dst_city, dst_city)
                
            if len(prev_str_date) > 1:
                str_date_diff = date_difference(str_date, prev_str_date)
            if len(prev_end_date) > 1:
                end_date_diff = date_difference(end_date, prev_end_date)

            print()
            print(t["author"])
            print(t["text"])                
            print(or_city_diff, dst_city_diff, str_date_diff, end_date_diff)
            
            final_vector.append([t["author"], t["text"], or_city_diff+dst_city_diff, str_date_diff+end_date_diff])
    
    return final_vector

ImportError: No module named 'rdflib'

In [ ]:

def gen_chat_data():
    chat_path = r"./Frames-dataset/chats"
    g = load_graph()
    
    chats = {}
    d = build_label_dict()
    for filename in os.listdir(chat_path):
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        frames_vector = prepare_frames_vector(chat,g)
        turns = []
        if 'turns' in chat:
            tsp = chat['turns'][0]['timestamp']
       
            for turn in chat['turns']:
               
                ts = turn['timestamp'] - tsp
                tsp = turn['timestamp']
                turns.append({"ti":ts,"text":turn["text"],"author":turn["author"]})

        chats[filename[:-5]] = {}
        chats[filename[:-5]]["turns"] = turns
        chats[filename[:-5]]["label"] = d[filename[:-5]]
        chats[filename[:-5]]["label"]
    return chats


In [ ]:
import numpy as np
# import pandas as pd
import _pickle as cPickle
from collections import defaultdict
import re

from numpy.linalg import norm

import sys
import os

os.environ['KERAS_BACKEND']='theano'

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers.merge import concatenate
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers
import theano

import functools
import nltk
nltk.download('punkt')
from nltk import tokenize

MAX_SENT_LENGTH = 40
MAX_SENTS = 20
MAX_NB_WORDS = 20000
MAX_TURNS = 60
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.1
TEST_SPLIT = 0.2
RNN_DIM = 50
GLOVE_DIR = "./data/glove"

In [ ]:
def create_texts_labels(chats,flag=False,length=0):
    texts=[]
    labels=[]
    for idx in chats.keys():
        if flag:
        
            text = "\n".join([x["text"] for x in chats[idx]["turns"][:length]])
        else:
            text = "\n".join([x["text"] for x in chats[idx]["turns"]])
        texts.append(text)
        sentences = tokenize.sent_tokenize(text)
        labels.append(chats[idx]["label"])   
    return texts,labels

In [ ]:
def create_data_with_turns(chats, flag=False,length=0):
    texts = []
    labels=[]
    chats_txt=[]
    vecs=[]
    for idx in chats.keys():
        if flag:
            text = "\n".join([x["text"] for x in chats[idx]["turns"][:length]])
            
        else:
            text = "\n".join([x["text"] for x in chats[idx]["turns"]])
        turns=[]
        vec=[]
        for turn in chats[idx]["turns"]:
            vec.append([len(turn["text"]),turn["ti"],0 if turn["author"].lower()=="wizard" else 1])
            texts.append(turn["text"])
            sentences = tokenize.sent_tokenize(turn["text"])
            turns.append(sentences)
        vecs.append(vec)
        chats_txt.append(turns)
        labels.append(chats[idx]["label"])
        

    return vecs


In [ ]:
def prepare_datasets(chats,flag=False, length =0):
    if flag:
        chats = {k: v for k, v in chats.items() if len(v["turns"])>length}
    
    texts,labels = create_texts_labels(chats,flag,length)
    vecs = create_data_with_turns(chats, flag,length)

    aux_data = np.zeros((len(texts), MAX_TURNS,3), dtype='float32')
    for i,vec in enumerate(vecs):
        if flag:
            for j,v in enumerate(vec[:length]):
                aux_data[i,j,:]= np.array(v)
        else:
            for j,v in enumerate(vec):
                aux_data[i,j,:]= np.array(v)
    
    
    
    norm_hlp = aux_data.max(axis=0).max(axis=0).astype(np.float)
    aux_data = aux_data.astype(np.float)/norm_hlp
#     print(norm_hlp.shape)
#     print(norm_hlp)
#     print(aux_data)
    labels = to_categorical(np.asarray(labels,dtype='float32'))

    print('Shape of label tensor:', labels.shape)
    print('Shape of aux tensor:', aux_data.shape)
#     data = data[:13]
#     labels = labels[:13]
    indices = np.arange(len(aux_data))
    np.random.shuffle(indices)
    aux_data = aux_data[indices]

    labels = labels[indices]
    nb_validation_samples = int(VALIDATION_SPLIT * aux_data.shape[0])
    nb_test_samples = int(TEST_SPLIT * aux_data.shape[0])

    x_train = aux_data[:-(nb_validation_samples+nb_test_samples)]
    y_train = labels[:-(nb_validation_samples+nb_test_samples)]
    x_val = aux_data[-(nb_validation_samples+nb_test_samples):-nb_test_samples]
    y_val = labels[-(nb_validation_samples+nb_test_samples):-nb_test_samples]
    x_test = aux_data[-nb_test_samples:]
    y_test = labels[-nb_test_samples:]
    
    return x_train, y_train,x_val, y_val,x_test,y_test

In [ ]:
def evaluate_model(model,x_train, y_train,x_val, y_val,x_test,y_test):
    print(model.summary())
    model.fit(x_train, y_train, validation_data=(x_val, y_val),
    nb_epoch=10, batch_size=50)

    return model.evaluate(x_test,y_test,batch_size=50, verbose=1, sample_weight=None)



In [ ]:
def rnn_network():

    review_input = Input(shape=(MAX_TURNS,3), dtype='float32')
    l_lstm = Bidirectional(LSTM(50))(review_input)


    preds = Dense(2, activation='softmax')(l_lstm)
    model = Model(review_input, preds)

    model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])
    return model

In [ ]:
  def run_net(net,file,x_train, y_train,x_val, y_val,x_test,y_test):
    model = net()
    res = evaluate_model(model,x_train, y_train,x_val, y_val,x_test,y_test)
    file.write(str(net)+"\n")
    file.write(str(res)+"\n")


In [ ]:
chats = gen_chat_data()
with open("rnnresults","w") as results_file:
    x_train, y_train,x_val, y_val,x_test,y_test = prepare_datasets(chats,True,4)
    run_net(rnn_network,results_file,x_train, y_train,x_val, y_val,x_test,y_test)
        